# LSA 潛在語意分析，使用 gensim

In [2]:
import os
from gensim import corpora, models, similarities

dictionary = corpora.Dictionary(document.split() for document in open('data/demo_lyrics_seg.txt'))

In [3]:
print(dictionary)

Dictionary(12 unique tokens: [u'\u5feb\u6a02', u'\u5de7\u514b\u529b', u'\u7406\u60f3', u'\u5496\u5561', u'\u98db\u7fd4']...)


In [4]:
texts = [[word for word in document.split()]
         for document in open('data/demo_lyrics_seg.txt')]
print(texts)

[['\xe5\xb7\xa7\xe5\x85\x8b\xe5\x8a\x9b', '\xe4\xb8\x96\xe7\x95\x8c', '\xe9\xa2\xa8\xe6\x99\xaf'], ['\xe6\x84\x9b\xe6\x83\x85', '\xe7\x94\x9c\xe7\xbe\x8e', '\xe7\x90\x86\xe6\x83\xb3', '\xe5\xb7\xa7\xe5\x85\x8b\xe5\x8a\x9b', '\xe5\xbf\xab\xe6\xa8\x82', '\xe6\xbb\x8b\xe5\x91\xb3'], ['\xe4\xb8\x96\xe7\x95\x8c', '\xe6\x84\x9b\xe6\x83\x85', '\xe5\x92\x96\xe5\x95\xa1', '\xe7\x94\x9c\xe7\xbe\x8e'], ['\xe6\x84\x9b\xe6\x83\x85', '\xe9\xa2\xa8\xe6\x99\xaf', '\xe6\x84\x9b\xe6\x83\x85', '\xe5\x92\x96\xe5\x95\xa1'], ['\xe6\x84\x9b\xe6\x83\x85', '\xe6\xbb\x8b\xe5\x91\xb3', '\xe7\x94\x9c\xe7\xbe\x8e'], ['\xe5\xa4\xa2\xe6\x83\xb3'], ['\xe9\x99\xbd\xe5\x85\x89', '\xe5\xa4\xa2\xe6\x83\xb3'], ['\xe9\x99\xbd\xe5\x85\x89', '\xe9\xa3\x9b\xe7\xbf\x94', '\xe5\xa4\xa2\xe6\x83\xb3'], ['\xe9\x99\xbd\xe5\x85\x89', '\xe7\x90\x86\xe6\x83\xb3', '\xe9\xa3\x9b\xe7\xbf\x94']]


In [5]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [6]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [7]:
print('debug 0')
print(lsi.show_topic(0, topn=20))
print('debug 1')
print(lsi.show_topic(1, topn=20))

debug 0
[(u'\u611b\u60c5', 0.71487716371695076), (u'\u751c\u7f8e', 0.40030631824144625), (u'\u5496\u5561', 0.28652980644694653), (u'\u6ecb\u5473', 0.27106193453225202), (u'\u5de7\u514b\u529b', 0.20578482989003349), (u'\u98a8\u666f', 0.20546545015982653), (u'\u4e16\u754c', 0.17742441113126872), (u'\u7406\u60f3', 0.17601679042327195), (u'\u5feb\u6a02', 0.15760480246795891), (u'\u967d\u5149', 0.025863037818709526), (u'\u98db\u7fd4', 0.023082088626059), (u'\u5922\u60f3', 0.0081152410401324048)]
debug 1
[(u'\u967d\u5149', 0.64890367236816204), (u'\u5922\u60f3', 0.51440997789451515), (u'\u98db\u7fd4', 0.46806292653010778), (u'\u7406\u60f3', 0.26261538395779432), (u'\u611b\u60c5', -0.091245447220822076), (u'\u5496\u5561', -0.082665317410177144), (u'\u98a8\u666f', -0.060709996377469884), (u'\u5feb\u6a02', 0.048155049319669338), (u'\u6ecb\u5473', 0.042076791584799315), (u'\u4e16\u754c', -0.042061470996758651), (u'\u5de7\u514b\u529b', 0.038101974337643611), (u'\u751c\u7f8e', 0.01006839557006634)

In [8]:
doc = "巧克力 世界 風景"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]

In [11]:
index = similarities.MatrixSimilarity(lsi[corpus], num_features=12)
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims[:9])

[(0, 0.99999994), (2, 0.99979514), (3, 0.99828625), (4, 0.99670351), (1, 0.96405917), (8, 0.052201528), (7, -0.074385986), (6, -0.080131441), (5, -0.093506016)]


In [12]:
doc = "愛情"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
index = similarities.MatrixSimilarity(lsi[corpus], num_features=12)
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims[:9])

[(2, 0.99999636), (0, 0.99984622), (3, 0.99915886), (4, 0.99512786), (1, 0.95925266), (8, 0.034684129), (7, -0.091859251), (6, -0.09759602), (5, -0.1109481)]


## 實戰

In [13]:
input_train_data_file = 'data/lyrics_word_net.dataset'
stop_word_file = 'data/stop_words.txt'

with open(stop_word_file) as f:
    stop_word_content = f.readlines()
stop_word_content = [x.strip() for x in stop_word_content]
stop_word_content = " ".join(stop_word_content)

dictionary = corpora.Dictionary(document.split() for document in open(input_train_data_file))
stoplist = set(stop_word_content.split())
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)
dictionary.compactify()

texts = [[word for word in document.split() if word not in stoplist]
         for document in open(input_train_data_file)]

In [14]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] >= 1]
         for text in texts]

corpus = [dictionary.doc2bow(text) for text in texts]

In [15]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=28)

In [16]:
doc = "沒有 山 不能 跨越 沒有 海 不能 冒險 讓 歷史 記得 這 一天 當我 用心 立下 諾言 沒有 事 不能 改變 沒有 夢 不能 實現 我 站 在 未來 最 前線 抬頭 迎接 每個 考驗 海闊天空 是 我 的 地圖 想 寫下 全新 紀錄 放眼 天下 在 等 我 去 征服 用 熱血 燃燒 黑夜 等待 最 燦爛 的 日出 看 陽光 與 我 賽跑 風雨 和 我 狂飆 我 的 驕傲 自己 打造 每個 夢 永遠 比天 高 一顆 心 為 希望 在 跳躍 讓 世界 為 我 歡呼 大地 為 我 炫耀 我 的 驕傲 你 會 看到 汗 和 淚 痛苦 的 煎熬 在 這 一刻 都 是 我 光榮 的 記號 海闊天空 是 我 的 地圖 想 寫下 全新 紀錄 放眼 天下 在 等 我 去 征服 用 熱血 燃燒 黑夜 等待 最 燦爛 的 日出 看 陽光 與 我 賽跑 風雨 和 我 狂飆 我 的 驕傲 自己 打造 每個 夢 永遠 比天 高 一顆 心 為 希望 在 跳躍 讓 世界 為 我 歡呼 大地 為 我 炫耀 我 的 驕傲 你 會 看到 汗 和 淚 ~ 痛苦 的 煎熬 在 這 一刻 都 是 我 ~ 光榮 的 記號 看 陽光 與 我 賽跑 風雨 和 我 狂飆 我 的 驕傲 自己 打造 每個 夢 ~ 永遠 比天 高 一顆 心 ~ 為 希望 在 跳躍 在 跳躍 ) 讓 世界 為 我 歡呼 大地 為 我 炫耀 我 的 驕傲 你 會 看到 你 會 看到 汗 和 淚 痛苦 的 煎熬 在 這 一刻 都 是 我 光榮 的 記號"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]

index = similarities.MatrixSimilarity(lsi[corpus], num_features=100)
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims[:5])

[(206, 0.99755716), (204, 0.95901793), (201, 0.95414102), (184, 0.94575286), (216, 0.94065785)]


In [17]:
lyrics = [];
fp = open("data/lyrics_word_net.dataset")
for i, line in enumerate(fp):
    lyrics.append(line)
fp.close()

url = [];
fp = open("data/lyrics_url.dataset")
for i, line in enumerate(fp):
    url.append(line)
fp.close()

for lyric in sims[:5]:
    print "\n相似歌詞：",  lyrics[lyric[0]]
    print "相似度：",  lyric[1]
    print "試聽連結：",  url[lyric[0]]


相似歌詞： 沒有 山 不能 跨越 沒有 海 不能 冒險 讓 歷史 記得 這 一天 當我 用心 立下 諾言 沒有 事 不能 改變 沒有 夢 不能 實現 我 站 在 未來 最 前線 抬頭 迎接 每個 考驗 海闊天空 是 我 的 地圖 想 寫 全新 紀錄 放眼 天下 在 等 我 去 征服 用 熱血 燃燒 黑夜 等 最 燦爛 的 日出 看 陽光 與 我 賽跑 風雨 和 我 狂奔 我 的 驕傲 自己 打造 每個 夢 永遠 比天 高 一顆 心 為 希望 在 跳躍 讓 世界 為 我 歡天喜地 大地 為 我 炫耀 我 的 驕傲 你 會 看見 汗水 和 淚水 痛苦 的 煎熬 在 這 一刻 都 是 我 光榮 的 記號 海闊天空 是 我 的 地圖 想 寫 全新 紀錄 放眼 天下 在 等 我 去 征服 用 熱血 燃燒 黑夜 等 最 燦爛 的 日出 看 陽光 與 我 賽跑 風雨 和 我 狂奔 我 的 驕傲 自己 打造 每個 夢 永遠 比天 高 一顆 心 為 希望 在 跳躍 讓 世界 為 我 歡天喜地 大地 為 我 炫耀 我 的 驕傲 你 會 看見 汗水 和 淚水 痛苦 的 煎熬 在 這 一刻 都 是 我 光榮 的 記號 看 陽光 與 我 賽跑 風雨 和 我 狂奔 我 的 驕傲 自己 打造 每個 夢 永遠 比天 高 一顆 心 為 希望 在 跳躍 在 跳躍 讓 世界 為 我 歡天喜地 大地 為 我 炫耀 我 的 驕傲 你 會 看見 你 會 看見 汗水 和 淚水 痛苦 的 煎熬 在 這 一刻 都 是 我 光榮 的 記號

相似度： 0.997557
試聽連結： http://tw.kkbox.com/song/JnBsJtc06MPXHPBIfPBIf0P4-index.html


相似歌詞： 當 我 和 世界 不 一樣 那 就讓 我 不 一樣 執著 對 我 來說 就是 以 剛克剛 我 如果 對 自己 妥協 如果 對 自己 說謊 即使 別人 原諒 我 也 不能 原諒 最美 的 願望 一定 最 瘋狂 我 就是 我 自己 的 神 在 我活 的 地方 我 和 我 最後 的 倔強 握緊 雙手 絕對 不 放 下 一站 是不是 天堂 就算 失望 不能 絕望 我 和 我 驕傲 的 倔強 我 在 風中 大聲 的 唱 這 一次 為 自己 瘋狂 就 這 一次 我 和 我 的 倔強 對 愛我 的 人別 緊張 我 的